In [11]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import json
import jsonlines
import pprint
import time
import pandas as pd
import numpy as np

In [36]:
NEWS_LIST_URL = "https://www.bigkinds.or.kr/api/news/search.do"

NEWS_LIST_PAYLOAD = {"indexName":"news",
"searchKey":"암호화폐",
"searchKeys":[{}],
"byLine":"",
"searchFilterType":"1",
"searchScopeType":"1",
"searchSortType":"date",
"sortMethod":"date",
"mainTodayPersonYn":"",
"startDate":"2023-05-25",
"endDate":"2023-08-25",
"newsIds":None,
"categoryCodes":[],
"providerCodes":["01100101","01100201","01100301","01100401","01100501","01100611","01100701","01100801","01100901","01101001","01101101","02100101","02100201","02100311","02100801","02100851","02100501","02100601","02100701"],
"incidentCodes":[],
"networkNodeType":"",
"topicOrigin":"",
"dateCodes":[],
"editorialIs":False,
"startNo":2,
"resultNumber":10,
"isTmUsable":False,
"isNotTmUsable":False}

NEWS_LIST_HEADERS = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Content-Length": "661",
    "Content-Type": "application/json;charset=UTF-8",
    "Cookie": "Bigkinds=32265CAF60809331F0F8E6D3FEF20EB7; _gid=GA1.3.1237921437.1693188567; _ga=GA1.3.285007105.1692864667; _gat=1; _ga_QWY27BS8JM=GS1.1.1693188567.8.1.1693188645.60.0.0",
    "Host": "www.bigkinds.or.kr",
    "Origin": "https://www.bigkinds.or.kr",
    "Referer": "https://www.bigkinds.or.kr/v2/news/index.do",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "sec-ch-ua": '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows"
}

# response = requests.request("POST", 
#                                 NEWS_LIST_URL, 
#                                 headers=NEWS_LIST_HEADERS, 
#                                 data = json.dumps(NEWS_LIST_PAYLOAD))

In [6]:
response

<Response [200]>

In [37]:
def get_news_list_by_index(start_index=1, result_len=1000):
    NEWS_LIST_PAYLOAD.update({"startNo":start_index, "resultNumber":str(result_len)})
    response = requests.request("POST", 
                                NEWS_LIST_URL, 
                                headers=NEWS_LIST_HEADERS, 
                                data = json.dumps(NEWS_LIST_PAYLOAD))
    return response.json()


def get_news_detail(NEWS_ID):
    url = f"https://www.bigkinds.or.kr/news/detailView.do?docId={NEWS_ID}&returnCnt=1"
    response = requests.request("GET", url)
    
    # 응답 상태 코드 확인
    if response.status_code != 200:
        print(f"Error for NEWS_ID={NEWS_ID}: {response.status_code}")
        print(response.text)  # 응답 내용 출력
        return None

    try:
        return response.json()
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError for NEWS_ID={NEWS_ID}: {e}")
        print(response.text)  # 에러 발생 시, 응답 내용 출력
        return None

# 샘플 NEWS_ID로 함수 테스트 (실제 실행 환경에서는 주석 처리 추천)
# sample_news_id = df_total.NEWS_ID.iloc[0].astype(int)
# get_news_detail(sample_news_id)


def write_jsonl(JSON_ARR, filename='output.jsonl'):
    with jsonlines.open(filename, mode='a') as writer:
        writer.write_all(JSON_ARR)

In [38]:
dfArr = []
for i in range(1,101):
    result = get_news_list_by_index(i)
    dfArr.append(pd.json_normalize(result["resultList"]))

In [39]:
df_total = pd.concat(dfArr)

In [40]:
df_total.shape

(635, 24)

In [41]:
df_total.reset_index(drop=True, inplace=True)

In [51]:
df_total['NEWS_ID'] = df_total['NEWS_ID'].astype(str)

In [46]:
search = '암호화폐'

In [47]:
#데이터 프레임 저장
now = datetime.datetime.now() 
df_total.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

In [59]:
get_news_detail(02100101.20230825170504001)

JSONDecodeError: Expecting value: line 9 column 1 (char 16)

In [56]:
list(df_total.NEWS_ID.items())

[(0, '02100101.20230825170504001'),
 (1, '02100501.20230825094135001'),
 (2, '02100311.20230824192036001'),
 (3, '02100851.20230824142006001'),
 (4, '01100801.20230824103202001'),
 (5, '02100851.20230824070806001'),
 (6, '02100801.20230824063138001'),
 (7, '02100201.20230824043254001'),
 (8, '01100801.20230824031008001'),
 (9, '02100601.20230823180319001'),
 (10, '01100611.20230823164825001'),
 (11, '02100311.20230823145551001'),
 (12, '02100601.20230823115029001'),
 (13, '01100501.20230823110305001'),
 (14, '02100601.20230823095305001'),
 (15, '02100201.20230823093023001'),
 (16, '01100611.20230823180243001'),
 (17, '02100851.20230821065636001'),
 (18, '01100901.20230821000829001'),
 (19, '02100201.20230819130038002'),
 (20, '02100701.20230819073032001'),
 (21, '02100501.20230819071258001'),
 (22, '02100701.20230819055255001'),
 (23, '02100201.20230819044939001'),
 (24, '02100201.20230819043117001'),
 (25, '02100851.20230818173851001'),
 (26, '01100201.20230818151335001'),
 (27, '0110

In [57]:
# 기사 데이터 jsonl 형식으로 저장
json_arr = []
for i, news_id in df_total.NEWS_ID.items():
    if news_id:
        res = get_news_detail(news_id)
        json_arr.append(res['detail'])
    if i % 100 == 0:
        print(i)
        write_jsonl(json_arr, 'result.jsonl')
        json_arr = []

JSONDecodeError: Expecting value: line 9 column 1 (char 16)

In [17]:
print("Response Content:", response.text[:500])  # 응답 내용의 첫 500자 출력                                

Response Content: {"getCategoryCodeList":[{"CategoryCode":"001000000","CategoryCount":"175","CategoryName":"정치"},{"CategoryCode":"002000000","CategoryCount":"315","CategoryName":"경제"},{"CategoryCode":"003000000","CategoryCount":"62","CategoryName":"사회"},{"CategoryCode":"004000000","CategoryCount":"9","CategoryName":"문화"},{"CategoryCode":"005000000","CategoryCount":"111","CategoryName":"국제"},{"CategoryCode":"006000000","CategoryCount":"3","CategoryName":"지역"},{"CategoryCode":"008000000","CategoryCount":"233","Cate


In [22]:
# 응답을 BeautifulSoup 객체로 변환
soup = BeautifulSoup(response.content, 'html.parser')

# 원하는 데이터 추출
# 예: 모든 기사 제목을 가져온다고 가정할 때
titles = soup.find_all('#news-results > div:nth-child(1) > div > div.cont > a > div > strong > span')  # 클래스 이름은 실제 웹사이트의 구조에 따라 다를 수 있습니다.
for title in titles:
    print(title.text)

In [27]:
# 응답으로 받은 JSON 데이터 파싱
data = response.json()

# 뉴스 기사의 내용만 추출 (기대되는 JSON 구조에 따라 변경될 수 있음)
articles = []
if 'result' in data and 'list' in data['result']:
    for item in data['result']['list']:
        if 'news' in item:
            articles.append(item['news'])

# articles 리스트에 뉴스 기사 내용이 저장됩니다.
print(articles)

[]


In [28]:
data

{'getCategoryCodeList': [{'CategoryCode': '001000000',
   'CategoryCount': '175',
   'CategoryName': '정치'},
  {'CategoryCode': '002000000', 'CategoryCount': '315', 'CategoryName': '경제'},
  {'CategoryCode': '003000000', 'CategoryCount': '62', 'CategoryName': '사회'},
  {'CategoryCode': '004000000', 'CategoryCount': '9', 'CategoryName': '문화'},
  {'CategoryCode': '005000000', 'CategoryCount': '111', 'CategoryName': '국제'},
  {'CategoryCode': '006000000', 'CategoryCount': '3', 'CategoryName': '지역'},
  {'CategoryCode': '008000000',
   'CategoryCount': '233',
   'CategoryName': 'IT_과학'}],
 'documentCount': 10,
 'totalCntNotAnalysis': 9,
 'errorMessage': None,
 'errorCode': None,
 'totalCntAnalysis': 626,
 'totalCount': 635,
 'getDateCodeList': [{'date': '2023', 'dateCount': '635'}],
 'getIncidentCodeList': [{'IncidentName': '범죄',
   'IncidentCount': '158',
   'IncidentCode': '1'},
  {'IncidentName': '사고', 'IncidentCount': '19', 'IncidentCode': '2'},
  {'IncidentName': '재해', 'IncidentCount': '4'